# Setting up Database POSTGRES PG ADMIN 4

In [ ]:
from dotenv import load_dotenv
import os, psycopg2
from openai import OpenAI

load_dotenv(r"C:\Point Detection\.env")

# Debug: print out the DSN we’re about to use
dsn = os.getenv("DATABASE_URL")
print("Using DATABASE_URL:", repr(dsn))

try:
    conn = psycopg2.connect(dsn)
    print("✅ Connected to Postgres!")
except Exception as e:
    print("❌ Connection failed:", e)
    raise

# If we get here, grab a cursor and close
cur = conn.cursor()
cur.execute("SELECT 1;")
print("Postgres replied:", cur.fetchone())
cur.close()
conn.close()


# Create the PostgreSQL Table for receipts

In [ ]:
#!/usr/bin/env python3
import os
from dotenv import load_dotenv
import psycopg2

# ─── Load environment variables ───
load_dotenv(r"C:\Point Detection\.env")  # adjust path as needed
dsn = os.getenv("DATABASE_URL")
if not dsn:
    raise RuntimeError("DATABASE_URL not set in .env")

# ─── Connect to Postgres ───
try:
    conn = psycopg2.connect(dsn)
    print("✅ Connected to Postgres")
except Exception as e:
    print("❌ Connection failed:", e)
    raise

# ─── Create cursor and table ───
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS receipts (
    receipt_no   TEXT PRIMARY KEY,
    shop         TEXT NOT NULL,
    settle_dt    DATE    NOT NULL,
    trans_time   TIMESTAMP NOT NULL,
    amount       REAL    NOT NULL
);
""")
conn.commit()
print("✅ Table `receipts` is ready in the database")

# ─── Clean up ───
cur.close()
conn.close()


# Link to Superbrandmall DATABASE

Check the store name and Type

In [1]:
#!/usr/bin/env python3
"""
test_fetch_stores.py

Fetch a small batch from the SBMCMS /store API
and dump out the full JSON so you can inspect all fields.
"""

import requests
import json

def main():
    url = "https://sbmcms-api.superbrandmall.com/store"
    params = {
        "page":  4,
        "limit": 5,
        "search": "",    # empty string => no filtering
        "lang":   "cn"
    }

    resp = requests.get(url, params=params, timeout=10)
    resp.raise_for_status()

    data = resp.json()
    print(json.dumps(data, indent=2, ensure_ascii=False))

if __name__ == "__main__":
    main()


{
  "status": "Success",
  "statusCode": 200,
  "result": {
    "data": [
      {
        "_id": "67b2bf01ea120c7cd1c3d969",
        "name": "程",
        "shortDescription": "‌程Studio：与知名的阿玛尼也有合作关系，被视为兄弟企业‌",
        "description": "‌程Studio是维沙企业旗下的品牌‌，与知名的阿玛尼也有合作关系，被视为兄弟企业‌。程Studio的品牌定位时尚在线，店面设计干净清爽，装修风格以白色为主，配以绿植点缀，营造出优雅且放松的环境‌",
        "type": "retail",
        "contactPhoneNumber": "15921867267",
        "locationId": "s_a15cd0c29e5660da",
        "floor": [
          "B1"
        ],
        "position": "B1F18C",
        "categoryId": "67aeed60c450717bfaaea597",
        "subCategoryId": "67aeed6fc450717bfaaea5a5",
        "isActive": true,
        "createdAt": "2025-03-19T19:47:11.855Z",
        "updatedAt": "2025-05-15T07:38:58.991Z",
        "coverImagePaths": [
          "https://cdn-cms-prod.superbrandmall.com/stores/cover/67b2bf01ea120c7cd1c3d969_0MpUARS2oF.avif",
          "https://cdn-cms-prod.superbrandmall.com/stores/cover/67b2bf01ea120c7cd1c3d969_dPoYYjuYJj.avif",
      

In [8]:
#!/usr/bin/env python3
"""
bilingual_store_extractor.py

Extract ALL stores with both Chinese and English names by calling the API
with both lang=cn and lang=en, then merge the results.
"""

import requests
import json
import time
from pathlib import Path
from collections import defaultdict

def fetch_stores_page(page, limit=50, lang="cn"):
    """Fetch a single page of stores from the API"""
    url = "https://sbmcms-api.superbrandmall.com/store"
    params = {
        "page": page,
        "limit": limit,
        "search": "",
        "lang": lang
    }
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7"
    }
    
    try:
        resp = requests.get(url, params=params, headers=headers, timeout=15)
        if resp.status_code == 200:
            data = resp.json()
            if isinstance(data, dict) and 'result' in data and 'data' in data['result']:
                return data['result']['data']
        return None
    except Exception as e:
        print(f"  Error fetching page {page} ({lang}): {e}")
        return None

def fetch_all_stores_by_language(lang="cn"):
    """Fetch all stores for a specific language"""
    all_stores = []
    page = 1
    
    print(f"Fetching all stores in {lang.upper()}...")
    
    while True:
        print(f"  Fetching page {page} ({lang})...")
        stores = fetch_stores_page(page, limit=50, lang=lang)
        
        if not stores:
            print(f"  No more stores found on page {page} ({lang})")
            break
        
        if len(stores) == 0:
            print(f"  Empty page {page} ({lang})")
            break
        
        all_stores.extend(stores)
        print(f"  Found {len(stores)} stores on page {page}. Total: {len(all_stores)}")
        
        # If we get less than the limit, we've reached the end
        if len(stores) < 50:
            print(f"  Reached end of data ({lang})")
            break
        
        page += 1
        time.sleep(0.3)  # Be respectful to the API
    
    print(f"Total {lang.upper()} stores: {len(all_stores)}")
    return all_stores

def merge_bilingual_data(cn_stores, en_stores):
    """Merge Chinese and English store data"""
    print("\nMerging Chinese and English data...")
    
    # Create lookup dictionary by store ID for English stores
    en_lookup = {store['_id']: store for store in en_stores}
    
    merged_stores = []
    matched_count = 0
    
    for cn_store in cn_stores:
        store_id = cn_store['_id']
        en_store = en_lookup.get(store_id)
        
        # Extract comprehensive store information
        merged_store = {
            "id": store_id,
            "name_chinese": cn_store.get('name', ''),
            "name_english": en_store.get('name', '') if en_store else '',
            "type": cn_store.get('type', ''),
            "description_chinese": cn_store.get('shortDescription', ''),
            "description_english": en_store.get('shortDescription', '') if en_store else '',
            "phone": cn_store.get('contactPhoneNumber', ''),
            "floor": cn_store.get('floor', []),
            "position": cn_store.get('position', ''),
            "price_range_yuan": {
                "min": cn_store.get('minPriceYuan'),
                "max": cn_store.get('maxPriceYuan')
            },
            "price_range_usd": {
                "min": cn_store.get('minPriceUsd'),
                "max": cn_store.get('maxPriceUsd')
            },
            "is_active": cn_store.get('isActive', False),
            "is_halal": cn_store.get('isHalal'),
            "is_spicy": cn_store.get('isSpicy'),
            "location_id": cn_store.get('locationId', ''),
            "category_id": cn_store.get('categoryId', ''),
            "sub_category_id": cn_store.get('subCategoryId', ''),
            "cover_images": cn_store.get('coverImagePaths', []),
            "environment_images": cn_store.get('environmentImagePaths', []),
            "created_at": cn_store.get('createdAt', ''),
            "updated_at": cn_store.get('updatedAt', ''),
            "has_english_data": bool(en_store)
        }
        
        if en_store:
            matched_count += 1
        
        merged_stores.append(merged_store)
    
    print(f"Merged {len(merged_stores)} stores")
    print(f"  - {matched_count} stores have both Chinese and English names")
    print(f"  - {len(merged_stores) - matched_count} stores have Chinese names only")
    
    return merged_stores

def create_summary_data(merged_stores):
    """Create summary with just names and types"""
    summary = []
    for store in merged_stores:
        summary.append({
            "id": store["id"],
            "name_chinese": store["name_chinese"],
            "name_english": store["name_english"],
            "type": store["type"],
            "is_active": store["is_active"]
        })
    return summary

def save_data(data, filename):
    """Save data to JSON file"""
    output_dir = Path("C:/Point Detection/superbrandmallstores")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    output_path = output_dir / filename
    
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print(f"✅ Saved {len(data)} items to: {output_path}")
        return True
    except Exception as e:
        print(f"❌ Error saving {filename}: {e}")
        return False

def generate_analysis(merged_stores):
    """Generate detailed analysis of the extracted data"""
    print(f"\n{'='*60}")
    print("DETAILED ANALYSIS")
    print('='*60)
    
    total = len(merged_stores)
    active_stores = len([s for s in merged_stores if s['is_active']])
    bilingual_stores = len([s for s in merged_stores if s['has_english_data']])
    
    print(f"Total stores: {total}")
    print(f"Active stores: {active_stores}")
    print(f"Stores with English names: {bilingual_stores}")
    print(f"Bilingual coverage: {bilingual_stores/total*100:.1f}%")
    
    # Count by type
    type_counts = defaultdict(int)
    for store in merged_stores:
        store_type = store.get("type", "Unknown")
        type_counts[store_type] += 1
    
    print(f"\nStores by type:")
    for store_type, count in sorted(type_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {store_type}: {count}")
    
    # Show sample bilingual stores
    print(f"\nSample bilingual stores:")
    bilingual_samples = [s for s in merged_stores if s['has_english_data']][:5]
    
    for i, store in enumerate(bilingual_samples, 1):
        print(f"  {i}. Chinese: {store['name_chinese']}")
        print(f"     English: {store['name_english']}")
        print(f"     Type: {store['type']}")
        print()
    
    # Show stores without English names
    chinese_only = [s for s in merged_stores if not s['has_english_data']]
    if chinese_only:
        print(f"Sample Chinese-only stores:")
        for i, store in enumerate(chinese_only[:3], 1):
            print(f"  {i}. {store['name_chinese']} ({store['type']})")

def main():
    """Main function to extract bilingual store data"""
    print("SBMCMS Bilingual Store Data Extractor")
    print("=" * 50)
    
    # Test API connectivity first
    print("Testing API connectivity...")
    test_stores = fetch_stores_page(1, 1, "cn")
    if not test_stores:
        print("❌ Cannot connect to API. Please check your connection.")
        return
    print("✅ API connection successful")
    
    # Fetch Chinese stores
    cn_stores = fetch_all_stores_by_language("cn")
    if not cn_stores:
        print("❌ Failed to fetch Chinese stores")
        return
    
    # Fetch English stores
    en_stores = fetch_all_stores_by_language("en")
    if not en_stores:
        print("⚠️ Failed to fetch English stores, continuing with Chinese only")
        en_stores = []
    
    # Merge the data
    merged_stores = merge_bilingual_data(cn_stores, en_stores)
    
    # Generate analysis
    generate_analysis(merged_stores)
    
    # Save complete data
    save_data(merged_stores, "complete_bilingual_stores.json")
    
    # Save summary data (names and types only)
    summary_data = create_summary_data(merged_stores)
    save_data(summary_data, "stores_summary_bilingual.json")
    
    # Save just the requested format (name_chinese, name_english, type)
    simple_format = []
    for store in merged_stores:
        simple_format.append({
            "name_chinese": store["name_chinese"],
            "name_english": store["name_english"],
            "type": store["type"]
        })
    save_data(simple_format, "stores_names_and_types.json")
    
    print(f"\n{'='*60}")
    print("EXTRACTION COMPLETED SUCCESSFULLY!")
    print(f"Files saved to: C:/Point Detection/superbrandmallstores/")
    print(f"- complete_bilingual_stores.json (full data)")
    print(f"- stores_summary_bilingual.json (summary)")
    print(f"- stores_names_and_types.json (names & types only)")
    print('='*60)

if __name__ == "__main__":
    main()

SBMCMS Bilingual Store Data Extractor
Testing API connectivity...
✅ API connection successful
Fetching all stores in CN...
  Fetching page 1 (cn)...
  Found 50 stores on page 1. Total: 50
  Fetching page 2 (cn)...
  Found 50 stores on page 2. Total: 100
  Fetching page 3 (cn)...
  Found 50 stores on page 3. Total: 150
  Fetching page 4 (cn)...
  Found 50 stores on page 4. Total: 200
  Fetching page 5 (cn)...
  Found 2 stores on page 5. Total: 202
  Reached end of data (cn)
Total CN stores: 202
Fetching all stores in EN...
  Fetching page 1 (en)...
  Found 50 stores on page 1. Total: 50
  Fetching page 2 (en)...
  Found 50 stores on page 2. Total: 100
  Fetching page 3 (en)...
  Found 50 stores on page 3. Total: 150
  Fetching page 4 (en)...
  Found 50 stores on page 4. Total: 200
  Fetching page 5 (en)...
  Found 2 stores on page 5. Total: 202
  Reached end of data (en)
Total EN stores: 202

Merging Chinese and English data...
Merged 202 stores
  - 202 stores have both Chinese and Engl

In [ ]:
#!/usr/bin/env python3
"""
clear_store_categories.py

Load DATABASE_URL from C:\Point Detection\.env, connect to Postgres,
and delete all rows from store_categories.
"""

from dotenv import load_dotenv
import os
import psycopg2

# 1) Load your .env
load_dotenv(r"C:\Point Detection\.env")

# 2) Get the connection string
dsn = os.getenv("DATABASE_URL")

# 3) Connect
conn = psycopg2.connect(dsn)
cur  = conn.cursor()

# 4) Delete all rows
cur.execute("TRUNCATE TABLE store_categories;")
conn.commit()
print("✅ All rows deleted from store_categories.")

# 5) Clean up
cur.close()
conn.close()


In [ ]:
import requests
import urllib3

# ─── Configuration ───────────────────────────────────────
LOGIN_PAGE = "https://hddc01.superbrandmall.com/pod-web/static/views/app/login.html"
LOGIN_API  = "https://hddc01.superbrandmall.com/pod-web/s/auth/0000/login?_dc=1752569108115"

# Replace these with your actual credentials
USER     = "cpit"
PASSWORD = "@abc1234"                  # use the MD5 hash if required
ORG_UUID = "0003"

# ─── Disable SSL warnings (self-signed certs) ───────────
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# ─── Create a session ────────────────────────────────────
session = requests.Session()
session.verify = False  # disable cert check
# If you need weak-DH support, you can mount a custom SSLAdapter here

# 1️⃣ Seed any initial cookies/CSRF
session.get(LOGIN_PAGE)

# 2️⃣ Login
payload = {
    "phone":       "",
    "messageCode": "",
    "userCode":    USER,
    "password":    PASSWORD,
    "orgUuid":     ORG_UUID,
}
headers = {
    "Content-Type": "application/json;charset=UTF-8",
    "Origin":       "https://hddc01.superbrandmall.com",
    "Referer":      "https://hddc01.superbrandmall.com/pod-web/static/index.html",
}

resp = session.post(LOGIN_API, json=payload, headers=headers)
resp.raise_for_status()

# 3️⃣ Print the jwt cookie
cookie_jar = session.cookies.get_dict()
print("Logged in! Cookies:", cookie_jar)
print("JWT token:", cookie_jar.get("jwt"))


# Fuzzy-matching from store_name with store_categories

In [5]:
import os
import json
import difflib
import psycopg2
from dotenv import load_dotenv

# ─── 0) (Re)connect ─────────────────────────────────────────────────
load_dotenv(r"C:\Point Detection\.env")
conn = psycopg2.connect(os.getenv("DATABASE_URL"))
cur  = conn.cursor()

# ─── 1) Load the parsed summary JSON ────────────────────────────────
summary_path = r"C:\Point Detection\preprocessing\processed\parsed_summary.json"
with open(summary_path, encoding="utf-8") as f:
    store_name = json.load(f)["store_name"]

# ─── 2) Fetch your in-DB map ─────────────────────────────────────────
cur.execute("SELECT store_name, bucket FROM store_categories;")
store_map   = dict(cur.fetchall())
store_keys  = list(store_map.keys())

# ─── 3) Fuzzy-match strictly ────────────────────────────────────────
cutoff  = 0.6
matches = difflib.get_close_matches(store_name, store_keys, n=1, cutoff=cutoff)
if not matches:
    raise ValueError(f"No match in store_categories for: {store_name!r}")

best   = matches[0]
bucket = store_map[best]

# ─── 4) Print your result ───────────────────────────────────────────
print(f"Extracted store_name: {store_name!r}")
print(f"Matched to:           {best!r} → Category: {bucket!r}")

# ─── 5) Clean up ────────────────────────────────────────────────────
cur.close()
conn.close()


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Point Detection\\preprocessing\\processed\\parsed_summary.json'

# Accuracy check

In [ ]:
# … assume store_name, best, store_map already defined …

from difflib import SequenceMatcher

ratio = SequenceMatcher(None, store_name, best).ratio()
bucket = store_map[best]

print(f"Extracted store_name: {store_name!r}")
print(f"Best match:           {best!r} → Category: {bucket!r}")
print(f"Match rate:           {ratio:.1%}")
